# MABe – Reservoir Computing Training

This notebook loads the preprocessed dataset and trains Reservoir Computing models.
It uses the data prepared by `02_dataset_processing_and_scaling.ipynb`.

In [1]:
%pip install reservoirpy scikit-learn matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from reservoirpy.nodes import Reservoir
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score, 
                           precision_recall_curve, auc, f1_score, precision_score, recall_score)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pickle
import json
import pandas as pd
from typing import Tuple, Dict, Any

# Set random seeds for reproducibility
np.random.seed(42)

def build_reservoir(units: int = 300, sr: float = 0.9, lr: float = 0.3, 
                   input_scaling: float = 0.5, seed: int = 42) -> Reservoir:
    """Build an Echo State Network reservoir with fixed parameters."""
    return Reservoir(
        units=units,
        sr=sr,     # spectral radius
        lr=lr,     # leaking rate
        input_scaling=input_scaling,
        seed=seed
    )

def reservoir_transform(reservoir: Reservoir, X_batch: np.ndarray, pooling: str = "last") -> np.ndarray:
    """Transform windowed time series through reservoir and pool states.
    
    Args:
        reservoir: Trained reservoir
        X_batch: (n_windows, window_size, n_features)
        pooling: "last" or "mean"
    
    Returns:
        Z: (n_windows, reservoir_units) pooled features
    """
    n_windows = X_batch.shape[0]
    reservoir_units = reservoir.units
    Z = np.zeros((n_windows, reservoir_units), dtype=np.float32)
    
    for i in range(n_windows):
        states = reservoir.run(X_batch[i])
        if pooling == "last":
            Z[i] = states[-1]  # Last state
        elif pooling == "mean":
            Z[i] = states.mean(axis=0)  # Mean across time
        else:
            raise ValueError(f"Unknown pooling: {pooling}")
        reservoir.reset()
    
    return Z

def create_groups_from_metadata(file_info: list, y_all: np.ndarray) -> np.ndarray:
    """Create groups array for GroupShuffleSplit based on video files."""
    groups = np.zeros(len(y_all), dtype=object)
    start_idx = 0
    
    for info in file_info:
        n_windows = info['n_windows']
        video_id = info['file'].strip()  # Remove any whitespace
        groups[start_idx:start_idx + n_windows] = video_id
        start_idx += n_windows
    
    return groups

def prepare_binary_labels(y_all: np.ndarray, none_id: int) -> np.ndarray:
    """Convert multi-class labels to binary: action (1) vs none (0)."""
    return (y_all != none_id).astype(int)

def prepare_action_labels(y_all: np.ndarray, none_id: int, action_classes: list, class_to_id: dict) -> Tuple[np.ndarray, np.ndarray]:
    """Extract action windows and their labels for stage B training/evaluation."""
    action_mask = y_all != none_id
    y_action = y_all[action_mask]
    
    # Convert to 0-3 range for 4 action classes
    action_id_to_local = {class_to_id[cls]: i for i, cls in enumerate(action_classes)}
    y_action_local = np.array([action_id_to_local[label] for label in y_action])
    
    return action_mask, y_action_local

def train_and_eval_stageA(features: np.ndarray, y_binary: np.ndarray, groups: np.ndarray, 
                         feature_name: str) -> Dict[str, Any]:
    """Train and evaluate Stage A: Action vs None detection."""
    
    # Group-based train/val split (80/20)
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, val_idx = next(gss.split(features, y_binary, groups))
    
    X_train, X_val = features[train_idx], features[val_idx]
    y_train, y_val = y_binary[train_idx], y_binary[val_idx]
    
    # Train logistic regression
    clf = LogisticRegression(
        solver="saga", 
        max_iter=5000, 
        class_weight="balanced",
        random_state=42
    )
    clf.fit(X_train, y_train)
    
    # Get probabilities for threshold tuning
    y_val_prob = clf.predict_proba(X_val)[:, 1]
    
    # Find best threshold using F-beta score (beta=2 favors recall)
    thresholds = np.linspace(0.01, 0.99, 99)
    fbeta_scores = []
    
    for thresh in thresholds:
        y_pred_thresh = (y_val_prob >= thresh).astype(int)
        # F-beta with beta=2 (favors recall)
        beta = 2
        prec = precision_score(y_val, y_pred_thresh, zero_division=0)
        rec = recall_score(y_val, y_pred_thresh, zero_division=0)
        if prec + rec > 0:
            fbeta = (1 + beta**2) * (prec * rec) / (beta**2 * prec + rec)
        else:
            fbeta = 0
        fbeta_scores.append(fbeta)
    
    best_idx = np.argmax(fbeta_scores)
    best_threshold = thresholds[best_idx]
    
    # Evaluate at best threshold
    y_val_pred = (y_val_prob >= best_threshold).astype(int)
    
    # Calculate PR-AUC
    precision_curve, recall_curve, _ = precision_recall_curve(y_val, y_val_prob)
    pr_auc = auc(recall_curve, precision_curve)
    
    results = {
        'feature_name': feature_name,
        'classifier': clf,
        'best_threshold': best_threshold,
        'pr_auc': pr_auc,
        'precision': precision_score(y_val, y_val_pred, zero_division=0),
        'recall': recall_score(y_val, y_val_pred, zero_division=0),
        'f1': f1_score(y_val, y_val_pred, zero_division=0),
        'fbeta': fbeta_scores[best_idx],
        'confusion_matrix': confusion_matrix(y_val, y_val_pred),
        'y_val_prob': y_val_prob,
        'y_val_true': y_val,
        'threshold_sweep': {
            'thresholds': thresholds,
            'fbeta_scores': fbeta_scores
        }
    }
    
    return results

def train_and_eval_stageB(features: np.ndarray, y_action: np.ndarray, groups: np.ndarray,
                         action_mask: np.ndarray, feature_name: str, action_classes: list) -> Dict[str, Any]:
    """Train and evaluate Stage B: Multi-class action classification."""
    
    # Only use features from action windows
    features_action = features[action_mask]
    
    # Group-based train/val split on action windows only
    action_groups = groups[action_mask]
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, val_idx = next(gss.split(features_action, y_action, action_groups))
    
    X_train, X_val = features_action[train_idx], features_action[val_idx]
    y_train, y_val = y_action[train_idx], y_action[val_idx]
    
    # Train logistic regression for multi-class
    clf = LogisticRegression(
        solver="saga",
        max_iter=5000,
        class_weight="balanced",
        multi_class="multinomial",
        random_state=42
    )
    clf.fit(X_train, y_train)
    
    # Predict
    y_val_pred = clf.predict(X_val)
    
    results = {
        'feature_name': feature_name,
        'classifier': clf,
        'classification_report': classification_report(y_val, y_val_pred, 
                                                     target_names=action_classes, 
                                                     output_dict=True, zero_division=0),
        'confusion_matrix': confusion_matrix(y_val, y_val_pred),
        'macro_f1': f1_score(y_val, y_val_pred, average='macro', zero_division=0),
        'y_val_pred': y_val_pred,
        'y_val_true': y_val
    }
    
    return results

## Load processed dataset

Load the windowed data created by the preprocessing notebook.

In [18]:
# Load processed dataset
PROCESSED_ROOT = Path("data/data_processed")

X_all = np.load(PROCESSED_ROOT / "X_windows.npy")
y_all = np.load(PROCESSED_ROOT / "y_windows.npy")

# Load class mappings
with open(PROCESSED_ROOT / "class_mappings.json", 'r') as f:
    class_mappings = json.load(f)

classes = class_mappings['classes']
class_to_id = class_mappings['class_to_id']
id_to_class = class_mappings['id_to_class']

with open(PROCESSED_ROOT / "file_info.pkl", 'rb') as f:
    file_info = pickle.load(f)

print("Dataset loaded:")
print(f"  X shape: {X_all.shape}")
print(f"  y shape: {y_all.shape}")
print(f"  Classes: {classes}")
print(f"  Class distribution:")
class_counts = np.bincount(y_all, minlength=len(classes))
for i, (cls, count) in enumerate(zip(classes, class_counts)):
    ratio = count / len(y_all)
    print(f"    {cls}: {count:,} ({ratio:.3f})")
print(f"  Files processed: {len(file_info)}")
print(f"  Total windows: {len(X_all)}")
print(f"  Features per timestep: {X_all.shape[2]}")

# Create groups for GroupShuffleSplit (one group per video)
groups = create_groups_from_metadata(file_info, y_all)
unique_videos = len(np.unique(groups))
print(f"  Unique video groups: {unique_videos}")

# Prepare labels for two-stage approach
none_id = class_to_id["none"]
action_classes = [cls for cls in classes if cls != "none"]

y_binary = prepare_binary_labels(y_all, none_id)
action_mask, y_action = prepare_action_labels(y_all, none_id, action_classes, class_to_id)

print(f"\nTwo-stage setup:")
print(f"  Stage A (binary): {y_binary.sum()}/{len(y_binary)} action windows ({y_binary.mean():.3f})")
print(f"  Stage B (4-class): {len(y_action)} action windows for classification")
print(f"  Action classes: {action_classes}")

Dataset loaded:
  X shape: (16395, 200, 10)
  y shape: (16395,)
  Classes: ['chase', 'avoid', 'attack', 'chaseattack', 'none']
  Class distribution:
    chase: 22 (0.001)
    avoid: 52 (0.003)
    attack: 111 (0.007)
    chaseattack: 27 (0.002)
    none: 16,183 (0.987)
  Files processed: 10
  Total windows: 16395
  Features per timestep: 10
  Unique video groups: 10

Two-stage setup:
  Stage A (binary): 212/16395 action windows (0.013)
  Stage B (4-class): 212 action windows for classification
  Action classes: ['chase', 'avoid', 'attack', 'chaseattack']


## Two-Stage Hierarchical Classification

Implement proper evaluation for extreme class imbalance.

## Two-Stage Hierarchical Classification

Implement proper evaluation for extreme class imbalance.

In [ ]:
# Reservoir parameters
RESERVOIR_PARAMS = {
    'units': 300,
    'sr': 0.9,
    'lr': 0.3,
    'input_scaling': 0.5,
    'seed': 42
}

# Build reservoir
reservoir = build_reservoir(**RESERVOIR_PARAMS)
print(f"Reservoir built: {RESERVOIR_PARAMS}")

# Prepare features
print("\nPreparing features...")

# Baseline features (flattened + standardized)
X_flat = X_all.reshape(X_all.shape[0], -1)
scaler_flat = StandardScaler()
X_flat_scaled = scaler_flat.fit_transform(X_flat)

# Reservoir features (last state pooling)
print("Computing reservoir features (last state pooling)...")
X_reservoir_last = reservoir_transform(reservoir, X_all, pooling="last")
scaler_reservoir = StandardScaler()
X_reservoir_last_scaled = scaler_reservoir.fit_transform(X_reservoir_last)

# Reservoir features (mean pooling)
reservoir.reset()  # Reset for second pass
print("Computing reservoir features (mean pooling)...")
X_reservoir_mean = reservoir_transform(reservoir, X_all, pooling="mean")
X_reservoir_mean_scaled = scaler_reservoir.fit_transform(X_reservoir_mean)

print("Features prepared:")
print(f"  Baseline: {X_flat_scaled.shape}")
print(f"  Reservoir (last): {X_reservoir_last_scaled.shape}")
print(f"  Reservoir (mean): {X_reservoir_mean_scaled.shape}")

# Define feature sets for comparison
feature_sets = {
    'Baseline': X_flat_scaled,
    'Reservoir_Last': X_reservoir_last_scaled,
    'Reservoir_Mean': X_reservoir_mean_scaled
}

print("\n" + "="*60)
print("TWO-STAGE HIERARCHICAL CLASSIFICATION")
print("="*60)

# Stage A: Action vs None Detection
print("\n" + "-"*40)
print("STAGE A: Action vs None Detection")
print("-"*40)

stageA_results = {}
for feature_name, features in feature_sets.items():
    print(f"\nTraining {feature_name}...")
    results = train_and_eval_stageA(features, y_binary, groups, feature_name)
    stageA_results[feature_name] = results
    
    print(f"  PR-AUC: {results['pr_auc']:.3f}")
    print(f"  Best threshold: {results['best_threshold']:.3f}")
    print(f"  Precision: {results['precision']:.3f}")
    print(f"  Recall: {results['recall']:.3f}")
    print(f"  F1: {results['f1']:.3f}")
    print(f"  F-beta (β=2): {results['fbeta']:.3f}")

# Stage B: Multi-class Action Classification
print("\n" + "-"*40)
print("STAGE B: Multi-class Action Classification")
print("-"*40)

stageB_results = {}
for feature_name, features in feature_sets.items():
    print(f"\nTraining {feature_name}...")
    results = train_and_eval_stageB(features, y_action, groups, action_mask, feature_name, action_classes)
    stageB_results[feature_name] = results
    
    print(f"  Macro F1: {results['macro_f1']:.3f}")
    print("  Per-class F1:")
    for cls in action_classes:
        f1 = results['classification_report'][cls]['f1-score']
        print(f"    {cls}: {f1:.3f}")

print("\n" + "="*60)
print("SUMMARY AND VISUALIZATIONS")
print("="*60)

# Summary table
print("\nSTAGE A RESULTS (Action Detection):")
print("Feature Set      | PR-AUC | Threshold | Precision | Recall | F1    | F-β(2)")
print("-" * 75)
for name, results in stageA_results.items():
    print(f"{name:<15} | {results['pr_auc']:<6.3f} | {results['best_threshold']:<9.3f} | {results['precision']:<9.3f} | {results['recall']:<6.3f} | {results['f1']:<5.3f} | {results['fbeta']:.3f}")

print("\nSTAGE B RESULTS (Action Classification):")
print("Feature Set      | Macro F1 | Chase | Avoid | Attack | ChaseAttack")
print("-" * 70)
for name, results in stageB_results.items():
    f1_scores = [results['classification_report'][cls]['f1-score'] for cls in action_classes]
    f1_str = " | ".join(f"{f1:.3f}" for f1 in f1_scores)
    print(f"{name:<15} | {results['macro_f1']:<8.3f} | {f1_str}")

# Visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Stage A: Confusion matrices
for i, (name, results) in enumerate(stageA_results.items()):
    cm = results['confusion_matrix']
    axes[i].imshow(cm, cmap='Blues')
    axes[i].set_title(f'Stage A: {name}')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('True')
    axes[i].set_xticks([0, 1])
    axes[i].set_yticks([0, 1])
    axes[i].set_xticklabels(['None', 'Action'])
    axes[i].set_yticklabels(['None', 'Action'])
    
    # Add text
    for x in range(2):
        for y in range(2):
            axes[i].text(y, x, cm[x, y], ha='center', va='center')

# Stage B: Confusion matrices
for i, (name, results) in enumerate(stageB_results.items()):
    cm = results['confusion_matrix']
    axes[i+3].imshow(cm, cmap='Blues')
    axes[i+3].set_title(f'Stage B: {name}')
    axes[i+3].set_xlabel('Predicted')
    axes[i+3].set_ylabel('True')
    axes[i+3].set_xticks(range(len(action_classes)))
    axes[i+3].set_yticks(range(len(action_classes)))
    axes[i+3].set_xticklabels(action_classes, rotation=45, ha='right')
    axes[i+3].set_yticklabels(action_classes)
    
    # Add text
    for x in range(len(action_classes)):
        for y in range(len(action_classes)):
            axes[i+3].text(y, x, cm[x, y], ha='center', va='center')

plt.tight_layout()
plt.show()

# Threshold sweep for best Stage A model
best_stageA = max(stageA_results.items(), key=lambda x: x[1]['fbeta'])
best_name, best_results = best_stageA

print(f"\nTHRESHOLD SWEEP for {best_name} (Stage A):")
thresholds = best_results['threshold_sweep']['thresholds']
fbeta_scores = best_results['threshold_sweep']['fbeta_scores']

# Find top 5 thresholds
top_indices = np.argsort(fbeta_scores)[-5:][::-1]
print("Top 5 thresholds by F-β(2) score:")
for i, idx in enumerate(top_indices):
    thresh = thresholds[idx]
    fbeta = fbeta_scores[idx]
    print(f"  {i+1}. Threshold {thresh:.3f}: F-β(2) = {fbeta:.3f}")

# Action detection statistics
total_action_windows = y_binary.sum()
detected_by_best = (best_results['y_val_prob'] >= best_results['best_threshold']).sum()
print(f"\nACTION DETECTION STATISTICS:")
print(f"  Total action windows in validation: {total_action_windows}")
print(f"  Detected by {best_name}: {detected_by_best}")
print(f"  Detection rate: {detected_by_best/total_action_windows:.3f}")

SyntaxError: invalid syntax (1065872545.py, line 171)